In [13]:
import os
import pandas as pd
from PIL import Image

import wandb

from ipywidgets import IntProgress

import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader


import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from time import time

import torch
import torch.utils.data as data

import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import albumentations as A
import cv2
import albumentations.pytorch

import matplotlib.pyplot as plt
import seaborn as sns
import random

In [14]:
data_dir = '/opt/ml/input/data/train'
img_dir = f'{data_dir}/images'
df_path = f'{data_dir}/train.csv'

In [15]:
def seed_everything(seed):
    """
    동일한 조건으로 학습을 할 때, 동일한 결과를 얻기 위해 seed를 고정시킵니다.
    
    Args:
        seed: seed 정수값
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
seed_everything(42)

In [16]:
df = pd.read_csv(df_path)
df.head()

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59


In [17]:
train_data_v1 = pd.read_csv("./full_labels.csv")
train_data_v1.rename(columns={'class':'label'},inplace=True)
train_data_v1

,file,gender,age,mask,map_age,label
0,/opt/ml/input/data/train/images/000001_female_...,1,45,0,1,4
1,/opt/ml/input/data/train/images/000001_female_...,1,45,1,1,10
2,/opt/ml/input/data/train/images/000001_female_...,1,45,0,1,4
3,/opt/ml/input/data/train/images/000001_female_...,1,45,0,1,4
4,/opt/ml/input/data/train/images/000001_female_...,1,45,2,1,16
...,...,...,...,...,...,...
18895,/opt/ml/input/data/train/images/006959_male_As...,0,19,0,0,0
18896,/opt/ml/input/data/train/images/006959_male_As...,0,19,0,0,0
18897,/opt/ml/input/data/train/images/006959_male_As...,0,19,2,0,12
18898,/opt/ml/input/data/train/images/006959_male_As...,0,19,0,0,0


In [18]:
train_data_v2 = pd.read_csv("./train_label.csv").drop(columns="Unnamed: 0")
train_data_v2.drop(columns=['race','gender','age','mask','id'],inplace=True)
train_data_v2

,path
0,000001_female_Asian_45/normal.jpg
1,000001_female_Asian_45/mask1.jpg
2,000001_female_Asian_45/mask2.jpg
3,000001_female_Asian_45/mask3.jpg
4,000001_female_Asian_45/mask4.jpg
...,...
18895,006959_male_Asian_19/mask2.jpg
18896,006959_male_Asian_19/mask3.jpg
18897,006959_male_Asian_19/mask4.jpg
18898,006959_male_Asian_19/mask5.jpg


In [19]:
train_data = pd.concat([train_data_v1,train_data_v2],axis=1)
train_data

,file,gender,age,mask,map_age,label,path
0,/opt/ml/input/data/train/images/000001_female_...,1,45,0,1,4,000001_female_Asian_45/normal.jpg
1,/opt/ml/input/data/train/images/000001_female_...,1,45,1,1,10,000001_female_Asian_45/mask1.jpg
2,/opt/ml/input/data/train/images/000001_female_...,1,45,0,1,4,000001_female_Asian_45/mask2.jpg
3,/opt/ml/input/data/train/images/000001_female_...,1,45,0,1,4,000001_female_Asian_45/mask3.jpg
4,/opt/ml/input/data/train/images/000001_female_...,1,45,2,1,16,000001_female_Asian_45/mask4.jpg
...,...,...,...,...,...,...,...
18895,/opt/ml/input/data/train/images/006959_male_As...,0,19,0,0,0,006959_male_Asian_19/mask2.jpg
18896,/opt/ml/input/data/train/images/006959_male_As...,0,19,0,0,0,006959_male_Asian_19/mask3.jpg
18897,/opt/ml/input/data/train/images/006959_male_As...,0,19,2,0,12,006959_male_Asian_19/mask4.jpg
18898,/opt/ml/input/data/train/images/006959_male_As...,0,19,0,0,0,006959_male_Asian_19/mask5.jpg


In [20]:
img_path = train_data['path']
labels = train_data['label']

In [46]:
class TrainDataset(Dataset):
    def __init__(self, img_path, labels, transfrom):
        self.img_path = img_path
        self.labels = labels
        self.transform = transfrom

    def __getitem__(self,idx):
        x = Image.open(self.img_path[idx])
        y = self.labels[idx]

        if self.transform:
            x = self.transform(x)

        return x,y

    def __len__(self):
        return len(self.img_path)

In [22]:
x_train, x_valid, y_train, y_valid = train_test_split(img_path, labels, test_size=0.2)
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
x_valid = x_valid.reset_index(drop=True)
y_valid = y_valid.reset_index(drop=True)

In [23]:
train_transform = A.Compose([
    # transforms.CenterCrop(400),
    A.HorizontalFlip(p=0.5),
    # transforms.RandomRotation(rotate_limit=15),
    albumentations.pytorch.transforms.ToTensorV2(),
    A.Normalize([0.485,0.456,0.406],[0.229,0.224,0.220])
    ])

vaild_transform = A.Compose([
    # transforms.CenterCrop(380),
    A.HorizontalFlip(p=0.5),
    albumentations.pytorch.transforms.ToTensorV2(),
    A.Normalize([0.485,0.456,0.406],[0.229,0.224,0.220])
    ])

mask_train_dataset = TrainDataset(x_train, y_train, train_transform)
mask_valid_dataset = TrainDataset(x_valid, y_valid, vaild_transform)


In [24]:
from multiprocessing import cpu_count

num_workers = int(cpu_count()/2)

batch_size = 32

mask_train_dataloader = DataLoader(
    mask_train_dataset,
    batch_size = batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
    )

mask_valid_dataloader = DataLoader(
    mask_valid_dataset,
    batch_size=batch_size,
    )

In [25]:
# from efficientnet_pytorch import EfficientNet
# model = EfficientNet.from_pretrained('efficientnet-b4')


from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0')

Loaded pretrained weights for efficientnet-b0


In [26]:
def rand_bbox(size, lam): # size : [Batch_size, Channel, Width, Height]
    W = size[2] 
    H = size[3] 
    cut_rat = np.sqrt(1. - lam)  # 패치 크기 비율
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)  

   	# 패치의 중앙 좌표 값 cx, cy
    cx = np.random.randint(W)
    cy = np.random.randint(H)
		
    # 패치 모서리 좌표 값 
    bbx1 = 0
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = W
    bby2 = np.clip(cy + cut_h // 2, 0, H)
   
    return bbx1, bby1, bbx2, bby2

In [27]:
# class FocalLoss(nn.Module):
#     def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
#         super(FocalLoss, self).__init__()
#         self.alpha = alpha
#         self.gamma = gamma
#         self.logits = logits
#         self.reduce = reduce

#     def forward(self, inputs, targets):
        
#         nn.CrossEntropyLoss()
    
#         ce_loss = nn.CrossEntropyLoss()(inputs, targets)

#         pt = torch.exp(-ce_loss)
#         F_loss = self.alpha * (1-pt)**self.gamma * ce_loss

#         if self.reduce:
#             return torch.mean(F_loss)
#         else:
#             return F_loss

In [28]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1, weight = None):
        """if smoothing == 0, it's one-hot method
           if 0 < smoothing < 1, it's smooth method
        """
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.weight = weight
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        assert 0 <= self.smoothing < 1
        pred = pred.log_softmax(dim=self.dim)

        if self.weight is not None:
            pred = pred * self.weight.unsqueeze(0)   

        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [29]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

print(f"{device} is using")

model.to(device)

LEARNING_RATE = 0.0001
NUM_EPOCH = 20

loss_fn = torch.nn.CrossEntropyLoss()
# loss_fn = LabelSmoothingLoss(classes=18)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

dataloaders = {
    "train" : mask_train_dataloader,
    "valid" : mask_valid_dataloader
}


cuda:0 is using


In [38]:
TrainDataset[0]

TypeError: Parameters to generic types must be types. Got 0.

In [45]:
import wandb
os.environ['WANDB_NOTEBOOK_NAME'] = 'first_v1.ipynb'
# wandb.init(project='mask_or_nomask', entity='아이디',config={
#     "K-fold": 5,
#     "batch_size": batch_size,
#     "lr"        : LEARNING_RATE,
#     "epochs"    : num_epochs,
#     "backborn"  : 'efficientnet-b7',
#     "criterion_name" : 'Cross-entropy'})

wandb.init(project='mask_classification',entity='junhyuk93',config={"batch_size": batch_size,
                   "lr"        : LEARNING_RATE,
                   "epochs"    : NUM_EPOCH,
                   "backborn"  : "effcientnet-b4",
                   "criterion_name" : "Cross_entropy"})

wandb: You can sync this run to the cloud by running:
wandb: wandb sync /opt/ml/code/wandb/offline-run-20210901_013649-36yx6l7f


wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


In [ ]:
best_test_accuracy = 0.
best_test_loss = 0.

for epoch in range(NUM_EPOCH):
  n_iter = 0
  epoch_f1 = 0
  for phase in ["train", "valid"]:
    running_loss = 0.
    running_acc = 0.
    running_f1 = 0.
    if phase == "train":
      model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "valid":
      model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함
  
    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함


      ####


      if np.random.random()>0.5: # cutmix가 실행될 경우     
        lam = np.random.beta(0.7, 0.3)
        rand_index = torch.randperm(images.size()[0]).to(device)
        target_a = labels # 원본 이미지 label
        target_b = labels[rand_index] # 패치 이미지 label       
        bbx1, bby1, bbx2, bby2 = rand_bbox(images.size(), lam)
        images[:, :, bbx1:bbx2, bby1:bby2] = images[rand_index, :, bbx1:bbx2, bby1:bby2]
        lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (images.size()[-1] * images.size()[-2]))
        outputs = model(images)
        loss = loss_fn(outputs, target_a) * lam + loss_fn(outputs, target_b) * (1. - lam) # 패치 이미지와 원본 이미지의 비율에 맞게 
      
      else: # cutmix가 실행되지 않았을 경우
        outputs= model(images) 
        loss= loss_fn(outputs, labels)

      _, preds= torch.max(outputs, 1) 


      ####

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트
        
        epoch_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
        n_iter += 1


      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장
    #   running_f1 += f1_score(pred)
    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)
    epoch_f1 = epoch_f1/n_iter

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "valid" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "valid" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
      best_test_loss = epoch_loss
      # torch.save(model,'/opt/ml/cod.pth')
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-44-a22607beea74>", line 8, in __getitem__
    x = Image.open(self.img_path[idx])
  File "/opt/conda/lib/python3.8/site-packages/PIL/Image.py", line 2904, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: '001169_female_Asian_55/mask4.jpg'


In [ ]:
torch.save(model,'/opt/ml/code/Hello_TT.pth')